# Usando mas de una Observacion (features) 
**documento original:** 
https://colab.research.google.com/notebooks/mlcc/validation.ipynb?utm_source=mlcc&utm_campaign=colab-external&utm_medium=referral&utm_content=validation-colab&hl=en

## Validacion
#### Objetivos de aprendizaje
* Aprender el uso de multiples observaciones, para mejorar la efectividad del modelo
* Depurar problemas con los datos del modelo
* Utilizar un set de datos para verificar si el modelo esta sobre-ajustando (overfitting) con los datos de validacion

*Al igual que con otros ejemplos, estamos usando el set de datos de  [California housing data set](https://developers.google.com/machine-learning/crash-course/california-housing-data-description)*

### Inicio
*Primero* que todo, carguemos y perparemos los datos. Esta vez, vamos a trabajar con multiples observaciones, asi pues tenemos que moduladizar la logica para el preprocesamiento correcto de las observaciones:


In [2]:
## NOTE QUE ESTE LIBRO UTILZA PYTHON2 PARA PODER UTILIZAR TENSORFLOW
from __future__ import print_function

import math

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

california_housing_dataframe = pd.read_csv("./california_housing_train.csv", sep=",")

# california_housing_dataframe = california_housing_dataframe.reindex(
#     np.random.permutation(california_housing_dataframe.index))
california_housing_dataframe.describe()


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0
mean,-119.6,35.6,28.6,2643.7,539.4,1429.6,501.2,3.9,207300.9
std,2.0,2.1,12.6,2179.9,421.5,1147.9,384.5,1.9,115983.8
min,-124.3,32.5,1.0,2.0,1.0,3.0,1.0,0.5,14999.0
25%,-121.8,33.9,18.0,1462.0,297.0,790.0,282.0,2.6,119400.0
50%,-118.5,34.2,29.0,2127.0,434.0,1167.0,409.0,3.5,180400.0
75%,-118.0,37.7,37.0,3151.2,648.2,1721.0,605.2,4.8,265000.0
max,-114.3,42.0,52.0,37937.0,6445.0,35682.0,6082.0,15.0,500001.0


In [3]:
def preprocess_features(california_housing_dataframe):
  """Prepares input features from California housing data set.

  Args:
    california_housing_dataframe: A Pandas DataFrame expected to contain data
      from the California housing data set.
  Returns:
    A DataFrame that contains the features to be used for the model, including
    synthetic features.
  """
  selected_features = california_housing_dataframe[
    ["latitude",
     "longitude",
     "housing_median_age",
     "total_rooms",
     "total_bedrooms",
     "population",
     "households",
     "median_income"]]
  processed_features = selected_features.copy()
  # Crea nueva observacion sintetica (synthetic feature)
  processed_features["rooms_per_person"] = (
    california_housing_dataframe["total_rooms"] /
    california_housing_dataframe["population"])
  return processed_features

def preprocess_targets(california_housing_dataframe):
  """Prepares target features (i.e., labels) from California housing data set.

  Args:
    california_housing_dataframe: A Pandas DataFrame expected to contain data
      from the California housing data set.
  Returns:
    A DataFrame that contains the target feature.
  """
  output_targets = pd.DataFrame()
  # Scale the target to be in units of thousands of dollars.
  output_targets["median_house_value"] = (
    california_housing_dataframe["median_house_value"] / 1000.0)
  return output_targets

Para el **set de entrenamiento**, elejimos los primeros 12000 ejemplos, de un total de 17000


In [4]:
## Define el set de datos de entrenamiento
training_examples = preprocess_features(california_housing_dataframe.head(12000))
training_examples.describe()

,latitude,longitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,rooms_per_person
count,12000.0,12000.0,12000.0,12000.0,12000.0,12000.0,12000.0,12000.0,12000.0
mean,34.6,-118.5,27.5,2655.7,547.1,1476.0,505.4,3.8,1.9
std,1.6,1.2,12.1,2258.1,434.3,1174.3,391.7,1.9,1.3
min,32.5,-121.4,1.0,2.0,2.0,3.0,2.0,0.5,0.0
25%,33.8,-118.9,17.0,1451.8,299.0,815.0,283.0,2.5,1.4
50%,34.0,-118.2,28.0,2113.5,438.0,1207.0,411.0,3.5,1.9
75%,34.4,-117.8,36.0,3146.0,653.0,1777.0,606.0,4.6,2.3
max,41.8,-114.3,52.0,37937.0,5471.0,35682.0,5189.0,15.0,55.2


In [5]:
## Define el set de datos de objetivos (targets)
training_targets = preprocess_targets(california_housing_dataframe.head(12000))
training_targets.describe()

,median_house_value
count,12000.0
mean,198.0
std,111.9
min,15.0
25%,117.1
50%,170.5
75%,244.4
max,500.0


Para el **set de validacion**, elejiremos los ultimos 5000 ejemplos (12000 entrenamiento + 5000 validacion = 17000 total)


In [6]:
# Define set de validacion 
validation_examples = preprocess_features(california_housing_dataframe.tail(5000))
validation_examples.describe()

,latitude,longitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,rooms_per_person
count,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0
mean,38.1,-122.2,31.3,2614.8,521.1,1318.1,491.2,4.1,2.1
std,0.9,0.5,13.4,1979.6,388.5,1073.7,366.5,2.0,0.6
min,36.1,-124.3,1.0,8.0,1.0,8.0,1.0,0.5,0.1
25%,37.5,-122.4,20.0,1481.0,292.0,731.0,278.0,2.7,1.7
50%,37.8,-122.1,31.0,2164.0,424.0,1074.0,403.0,3.7,2.1
75%,38.4,-121.9,42.0,3161.2,635.0,1590.2,603.0,5.1,2.4
max,42.0,-121.4,52.0,32627.0,6445.0,28566.0,6082.0,15.0,18.3


In [7]:
# Define set de objetivos
validation_targets = preprocess_targets(california_housing_dataframe.tail(5000))
validation_targets.describe()

,median_house_value
count,5000.0
mean,229.5
std,122.5
min,15.0
25%,130.4
50%,213.0
75%,303.2
max,500.0


## Tarea 1 - Examinar los datos
De acuerdo a los datos anteriores, tenemos nueve (9) observaciones de entrada que podemos usar.

Estas son algunas observaciones / verificaciones que podemos hacer respecto a los datos:
* Para algunos valores, como median_house_value, podemos verificar si los valores estan dentro los rangos razonables 
* Para otros valores, como latitud y longitude, podemos verificar si corresponden al area esperada, e.g. usando Google search 

Mirando cerca, existen los siguientes anomalias:
* median_income esta en la escala de 3 a 15 (aproximadamente).  No es claro a que se refiere la escala de los valores. No esta documentado; podemos asumir que entre mas alto el numero, el ingreso es mayor
* El maximo valor de median_house_value es 500,001. Lo que se ve como un limite artificial de alguna clase.
* Nuestro calculo/observacion de rooms_per_person es en general en una escala que parece correcta, con el 75% con un valor cercano a 2. Aunque existen en los datos valores altos, como 18, lo que puede indicar que los datos no son exactos.

Utilizaremos las observaciones como vienen en el archivo. Pero recuerde que debe desarrollar una intuicion de revisar los datos y verificar si son consistentes con la informacion esperada.

